# Pyspark_Classification_Basic_Churn

In [1]:
#1 - import module
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [2]:
#2 - Create spark context
sc = SparkContext.getOrCreate()

In [3]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [4]:
sc.getConf().toDebugString()

u'spark.app.id=application_1541652388947_0026\nspark.app.name=pyspark-shell\nspark.driver.appUIAddress=http://hdp-all-spark-new.c.kao-project-v1.internal:4040\nspark.driver.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64\nspark.driver.host=hdp-all-spark-new.c.kao-project-v1.internal\nspark.driver.port=44448\nspark.eventLog.dir=hdfs:///spark2-history/\nspark.eventLog.enabled=true\nspark.executor.extraJavaOptions=-XX:+UseNUMA\nspark.executor.extraLibraryPath=/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64\nspark.executor.id=driver\nspark.executorEnv.PYTHONPATH=/usr/hdp/3.0.0.0-1634/spark2/python:/usr/hdp/3.0.0.0-1634/spark2/python/lib/py4j-0.10.7-src.zip:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.7-src.zip\nspark.history.fs.cleaner.enabled=true\nspark.history.fs.cleaner.interval=7d\nspark.history.fs.cleaner.maxAge=90d\nspark.history.fs.logDirectory=hdfs:///spark2-his

In [5]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.history.fs.cleaner.interval', u'7d'),
 (u'spark.shuffle.io.serverThreads', u'128'),
 (u'spark.yarn.historyServer.address',
  u'hdp-all-spark-new.c.kao-project-v1.internal:18081'),
 (u'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
  u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.driver.port', u'44448'),
 (u'spark.driver.host', u'hdp-all-spark-new.c.kao-project-v1.internal'),
 (u'spark.sql.statistics.fallBackToHdfs', u'true'),
 (u'spark.shuffle.file.buffer', u'1m'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.his

In [6]:
#3 - Setup SparkSession(SparkSQL)
spark = (SparkSession
         .builder
         .appName("Pyspark_Classification_Basic_Churn")
         .getOrCreate())
print spark

In [7]:
#4 - Read file to spark DataFrame
data = (spark
        .read
        .option("header","true")
        .option("inferSchema", "true")
        .csv("/user/admin/data/churn.csv"))
data.cache()
print "finish caching data"

finish caching data


In [9]:
#5 - Understand data and problems
category = ['International plan','Voice mail plan']
continuous = ['Number vmail messages','Total day minutes','Total day calls','Total day charge','Total eve minutes','Total eve calls','Total eve charge','Total night minutes','Total night calls','Total night charge','Total intl minutes','Total intl calls','Total intl charge','Customer service calls']
label = 'churn'

unique_features = ['State','Account length','Area code']
unused_features = ['Total day charge','Total eve charge','Total night charge','Total intl charge']
#bcz charges computed from minutes / 22.2252

print (len(category) + len(continuous))


16


In [11]:
data.describe().toPandas()

summary State      Account length           Area code International plan  \
0   count  3000                3000                3000               3000   
1    mean  None  101.09333333333333  437.04566666666665               None   
2  stddev  None  39.569843901693076   42.26102514103134               None   
3     min    AK                   1                 408                 No   
4     max    WY                 243                 510                Yes   

  Voice mail plan Number vmail messages  Total day minutes  \
0            3000                  3000               3000   
1            None                 8.011  179.5585000000005   
2            None    13.629996500422667  54.64951726833965   
3              No                     0                0.0   
4             Yes                    51              350.8   

      Total day calls    Total day charge  Total eve minutes  \
0                3000                3000               3000   
1  100.56133333333334  30.525506666666693  201.1894666666669   
2   20.05754238578949   9.290379267048115  51.20153176094548   
3                   0                 0.0                0.0   
4                 165               59.64              363.7   

      Total eve calls    Total eve charge Total night minutes  \
0                3000                3000                3000   
1  100.01833333333333  17.101323333333337  201.26770000000008   
2  19.975977500853205   4.352130347501413   50.48089883403788   
3                   0                 0.0                23.2   
4                 170               30.91               395.0   

    Total night calls Total night charge  Total intl minutes  \
0                3000               3000                3000   
1   99.97733333333333  9.057113333333342  10.263133333333347   
2  19.542009012918196  2.271682573422988  2.8103131739364486   
3                  33               1.04                 0.0   
4                 175              17.77                20.0   

     Total intl calls   Total intl charge Customer service calls  
0                3000                3000                   3000  
1               4.489   2.771566666666659     1.5616666666666668  
2  2.4672335547743733  0.7587626676487821     1.3206482976923197  
3                   0                 0.0                      0  
4                  20                 5.4                      9

In [12]:
data.printSchema()

root
 |-- State: string (nullable = true)
 |-- Account length: integer (nullable = true)
 |-- Area code: integer (nullable = true)
 |-- International plan: string (nullable = true)
 |-- Voice mail plan: string (nullable = true)
 |-- Number vmail messages: integer (nullable = true)
 |-- Total day minutes: double (nullable = true)
 |-- Total day calls: integer (nullable = true)
 |-- Total day charge: double (nullable = true)
 |-- Total eve minutes: double (nullable = true)
 |-- Total eve calls: integer (nullable = true)
 |-- Total eve charge: double (nullable = true)
 |-- Total night minutes: double (nullable = true)
 |-- Total night calls: integer (nullable = true)
 |-- Total night charge: double (nullable = true)
 |-- Total intl minutes: double (nullable = true)
 |-- Total intl calls: integer (nullable = true)
 |-- Total intl charge: double (nullable = true)
 |-- Customer service calls: integer (nullable = true)
 |-- Churn: boolean (nullable = true)



In [13]:
data.sample(False, 0.001, 1234).toPandas()

State  Account length  Area code International plan Voice mail plan  \
0    MD             106        510                 No              No   
1    KY             151        408                 No             Yes   
2    OH              74        415                 No              No   
3    NJ              96        510                 No              No   
4    NM             232        408                 No              No   
5    TX              71        415                 No             Yes   

   Number vmail messages  Total day minutes  Total day calls  \
0                      0              213.9               95   
1                     17              214.7               97   
2                      0              136.7              106   
3                      0              150.0              122   
4                      0              165.6              104   
5                     39              183.2              103   

   Total day charge  Total eve minutes  Total eve calls  Total eve charge  \
0             36.36              151.9               70             12.91   
1             36.50              138.5               90             11.77   
2             23.24              228.6              105             19.43   
3             25.50              218.5              116             18.57   
4             28.15              195.9              115             16.65   
5             31.14              209.4              111             17.80   

   Total night minutes  Total night calls  Total night charge  \
0                260.1                124               11.70   
1                169.1                 44                7.61   
2                265.3                114               11.94   
3                212.4                 89                9.56   
4                118.3                 77                5.32   
5                172.4                109                7.76   

   Total intl minutes  Total intl calls  Total intl charge  \
0                12.2                 5               3.29   
1                 8.6                 4               2.32   
2                 9.8                 4               2.65   
3                 9.8                 1               2.65   
4                11.8                 3               3.19   
5                11.9                 6               3.21   

   Customer service calls  Churn  
0                       3  False  
1                       1  False  
2                       0  False  
3                       3  False  
4                       1  False  
5                       1  False

In [14]:
data.groupBy(label).count().toPandas()

churn  count
0   True    438
1  False   2562

In [15]:
#6 - Change column type from boolean to string
data.select(label).printSchema()
data = data.withColumn(label, data[label].cast("string"))
data.select(label).printSchema()

root
 |-- churn: boolean (nullable = true)

root
 |-- churn: string (nullable = true)



In [16]:
#8 - Remove unused variables
print "number of features : " + str(len(data.drop(label).head()))
for unused_feature in unique_features + unused_features:
    print unused_feature
    data = data.drop(unused_feature)
print "\nnumber of features remain : " + str(len(data.drop(label).head()))


category = [feature for feature in category if feature not in (unique_features + unused_features)]
continuous = [feature for feature in continuous if feature not in (unique_features + unused_features)]

print "\nnumber of features remain : " + str(len(category) + len(continuous))

number of features : 19
State
Account length
Area code
Total day charge
Total eve charge
Total night charge
Total intl charge

number of features remain : 12

number of features remain : 12


In [17]:
#9 - String indexer - label
featureidx = StringIndexer(inputCol = label, outputCol='label')
string_indexer_model = featureidx.fit(data)
data_indexed = string_indexer_model.transform(data)

print type(featureidx)

print "\n"

data_indexed.select(label,'label').sample(False, 0.001, 1234).toPandas()

<class 'pyspark.ml.feature.StringIndexer'>




churn  label
0  false    0.0
1  false    0.0
2  false    0.0
3  false    0.0
4  false    0.0
5  false    0.0

In [18]:
#10 - String indexer - features
featureidx_list = [StringIndexer(inputCol = c, outputCol=c + 'idx') for c in category]
for featureidx in featureidx_list:
    string_indexer_model = featureidx.fit(data_indexed)
    data_indexed = string_indexer_model.transform(data_indexed)

print type(featureidx_list[0])

print "\n"

data_indexed.select('International plan','International planidx','Voice mail plan','Voice mail planidx').sample(False, 0.001, 1234).toPandas()

<class 'pyspark.ml.feature.StringIndexer'>




International plan  International planidx Voice mail plan  \
0                 No                    0.0              No   
1                 No                    0.0             Yes   
2                 No                    0.0              No   
3                 No                    0.0              No   
4                 No                    0.0              No   
5                 No                    0.0             Yes   

   Voice mail planidx  
0                 0.0  
1                 1.0  
2                 0.0  
3                 0.0  
4                 0.0  
5                 1.0

In [19]:
#11 - Create Vector
features = continuous + [c + 'idx' for c in category]
vector_assembler_model =  VectorAssembler(inputCols = features ,outputCol="features")
data_vector = vector_assembler_model.transform(data_indexed)
data_vector.cache()

print type(vector_assembler_model)

print "\n"

data_vector.select('features','label').sample(False, 0.001, 1234).toPandas()

<class 'pyspark.ml.feature.VectorAssembler'>




features  label
0  [0.0, 213.9, 95.0, 151.9, 70.0, 260.1, 124.0, ...    0.0
1  [17.0, 214.7, 97.0, 138.5, 90.0, 169.1, 44.0, ...    0.0
2  [0.0, 136.7, 106.0, 228.6, 105.0, 265.3, 114.0...    0.0
3  [0.0, 150.0, 122.0, 218.5, 116.0, 212.4, 89.0,...    0.0
4  [0.0, 165.6, 104.0, 195.9, 115.0, 118.3, 77.0,...    0.0
5  [39.0, 183.2, 103.0, 209.4, 111.0, 172.4, 109....    0.0

In [18]:
#12 - split Train and Test data
data_vector = data_vector.sort(label)
(trainingData, testData) = data_vector.randomSplit([0.7, 0.3],seed = 50)

print "data count : " + str(data_vector.count())
print "trainingData count : " + str(trainingData.count())
print "testData count : " + str(testData.count())

data_vector.groupBy('label').count().show()
trainingData.groupBy('label').count().show()
testData.groupBy('label').count().show()

data count : 3000
trainingData count : 2084
testData count : 916
+-----+-----+
|label|count|
+-----+-----+
|  0.0| 2562|
|  1.0|  438|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0| 1773|
|  1.0|  311|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|  789|
|  1.0|  127|
+-----+-----+



In [19]:
#13 - Create model
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

print dt


DecisionTreeClassifier_4473ad414659188984c0


In [20]:
#14 - Train model
model = dt.fit(trainingData)

In [21]:
#15 - Make predictions
predictions = model.transform(testData)
predictions.cache()

DataFrame[International plan: string, Voice mail plan: string, Number vmail messages: int, Total day minutes: double, Total day calls: int, Total eve minutes: double, Total eve calls: int, Total night minutes: double, Total night calls: int, Total intl minutes: double, Total intl calls: int, Customer service calls: int, churn: string, label: double, International planidx: double, Voice mail planidx: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [22]:
# Print sample result
predictions.toPandas()

International plan Voice mail plan  Number vmail messages  \
0                   No              No                      0   
1                   No              No                      0   
2                   No              No                      0   
3                   No              No                      0   
4                   No              No                      0   
5                   No              No                      0   
6                   No              No                      0   
7                   No              No                      0   
8                   No              No                      0   
9                   No              No                      0   
10                  No              No                      0   
11                  No              No                      0   
12                  No              No                      0   
13                  No              No                      0   
14                  No              No                      0   
15                  No              No                      0   
16                  No              No                      0   
17                  No              No                      0   
18                  No              No                      0   
19                  No              No                      0   
20                  No              No                      0   
21                  No              No                      0   
22                  No              No                      0   
23                  No              No                      0   
24                  No              No                      0   
25                  No              No                      0   
26                  No              No                      0   
27                  No              No                      0   
28                  No              No                      0   
29                  No              No                      0   
..                 ...             ...                    ...   
886                Yes              No                      0   
887                Yes              No                      0   
888                Yes              No                      0   
889                Yes              No                      0   
890                Yes              No                      0   
891                Yes              No                      0   
892                Yes              No                      0   
893                Yes              No                      0   
894                Yes              No                      0   
895                Yes              No                      0   
896                Yes              No                      0   
897                Yes              No                      0   
898                Yes              No                      0   
899                Yes              No                      0   
900                Yes              No                      0   
901                Yes              No                      0   
902                Yes              No                      0   
903                Yes              No                      0   
904                Yes             Yes                     18   
905                Yes             Yes                     20   
906                Yes             Yes                     21   
907                Yes             Yes                     24   
908                Yes             Yes                     26   
909                Yes             Yes                     28   
910                Yes             Yes                     29   
911                Yes             Yes                     29   
912                Yes             Yes                     32   
913                Yes             Yes                     32   
914                Yes             Yes                     42   
915                Yes             Yes 

In [24]:
# Print sample result
predictions.select("prediction", "rawPrediction", "probability", "label", "features").toPandas()

prediction   rawPrediction                                 probability  \
0           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
1           0.0   [198.0, 14.0]    [0.9339622641509434, 0.0660377358490566]   
2           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
3           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
4           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
5           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
6           0.0   [198.0, 14.0]    [0.9339622641509434, 0.0660377358490566]   
7           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
8           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
9           0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
10          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
11          0.0   [198.0, 14.0]    [0.9339622641509434, 0.0660377358490566]   
12          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
13          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
14          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
15          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
16          0.0   [198.0, 14.0]    [0.9339622641509434, 0.0660377358490566]   
17          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
18          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
19          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
20          0.0   [198.0, 14.0]    [0.9339622641509434, 0.0660377358490566]   
21          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
22          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
23          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
24          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
25          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
26          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
27          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
28          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
29          0.0  [1127.0, 29.0]  [0.9749134948096886, 0.025086505190311418]   
..          ...             ...                                         ...   
886         1.0     [1.0, 38.0]   [0.02564102564102564, 0.9743589743589743]   
887         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
888         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
889         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
890         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
891         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
892         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
893         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
894         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
895         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
896         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
897         1.0     [1.0, 27.0]   [0.03571428571428571, 0.9642857142857143]   
898         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
899         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
900         1.0     [0.0, 33.0]                                  [0.0, 1.0]   
901         1.0      [0.0, 7.0]                                  [0.0, 1.0]   
902         1.0     [0.0, 48.0]                                  [0.0, 1.0]   
903         1.0      [6.0, 8.0]   [0.42857142857142855, 0.5714285714285714]   
904         1.0     [0.0, 33.0]                        

In [25]:
#16 - Evaluate model
for metricName in ['accuracy','weightedPrecision','weightedRecall','f1']:
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName=metricName)
    result = evaluator.evaluate(predictions)
    print '%s = %g' % (metricName,result)
    if(metricName == 'accuracy') :
        print("error = %g " % (1.0 - result))


accuracy = 0.960699
error = 0.0393013 
weightedPrecision = 0.959687
weightedRecall = 0.960699
f1 = 0.959725


In [26]:
#17 - Show tree diagram
treeModel_debug_str = model.toDebugString
print treeModel_debug_str

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4473ad414659188984c0) of depth 5 with 53 nodes
  If (feature 1 <= 253.05)
   If (feature 9 <= 3.5)
    If (feature 10 in {1.0})
     If (feature 8 <= 2.5)
      Predict: 1.0
     Else (feature 8 > 2.5)
      If (feature 7 <= 12.95)
       Predict: 0.0
      Else (feature 7 > 12.95)
       Predict: 1.0
    Else (feature 10 not in {1.0})
     If (feature 1 <= 221.75)
      If (feature 2 <= 120.5)
       Predict: 0.0
      Else (feature 2 > 120.5)
       Predict: 0.0
     Else (feature 1 > 221.75)
      If (feature 3 <= 270.75)
       Predict: 0.0
      Else (feature 3 > 270.75)
       Predict: 1.0
   Else (feature 9 > 3.5)
    If (feature 1 <= 171.14999999999998)
     If (feature 3 <= 215.95)
      If (feature 2 <= 115.5)
       Predict: 1.0
      Else (feature 2 > 115.5)
       Predict: 1.0
     Else (feature 3 > 215.95)
      If (feature 1 <= 138.0)
       Predict: 1.0
      Else (feature 1 > 138.0)
       Predict: 0.0
    Els